# **Statistical learning final project**: modelling

### Daniel A.
### UID: 100444499

### Importing libraries and setting options

In [37]:
library(dplyr)
library(MASS)
library(e1071) 
library(VGAM)
library(caret)

In [38]:
# turn off warnings
options(warn=-1)

### Importing the data

In [39]:
df = read.csv('./data/data_2.csv')

### Preparing training and target sets

In [41]:
# vars
vars = names(df)
# training vars
train = vars[6:(length(vars)-2)]
train = df %>% dplyr::select(train)
# saving train to csv
write.csv(train,'./data/final/train.csv')

# target var
target = vars[length(vars)]
target = df %>% dplyr::select(target)

# saving target to csv
target$hdi_cat <- as.factor(target$hdi_cat)
write.csv(target,'./data/final/target.csv')

# target for models
dep <- as.factor(levels(target$hdi_cat))


In [42]:
# leaving a dataset with everything included except IDs
data <- cbind(train,target)

## Model 1: Logistic Regression (in R)

We use 10 reps of 10-fold CV

In [47]:
control <- trainControl(method = "repeatedcv",
                        repeats = 10,
                        number = 10 )

In [57]:
lrfit <- train(hdi_cat ~ ., 
               method = "glmnet",
               family = "multinomial",
               data = data,
               metric = "Accuracy",
               preProcess = c("center","scale"),
               tuneGrid = expand.grid(alpha = seq(0, 2, 0.1), 
                                      lambda = seq(0, .1, 0.01)),
               trControl = control)

In [58]:
lrpred <- predict(lrfit, data)

In [59]:
confusionMatrix(lrpred, data$hdi_cat)

Confusion Matrix and Statistics

           Reference
Prediction  high low medium very high
  high        48   0      7         8
  low          0  29      5         0
  medium       1   3     24         0
  very high    3   0      0        56

Overall Statistics
                                         
               Accuracy : 0.8533         
                 95% CI : (0.7937, 0.901)
    No Information Rate : 0.3478         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.7989         
                                         
 Mcnemar's Test P-Value : NA             

Statistics by Class:

                     Class: high Class: low Class: medium Class: very high
Sensitivity               0.9231     0.9062        0.6667           0.8750
Specificity               0.8864     0.9671        0.9730           0.9750
Pos Pred Value            0.7619     0.8529        0.8571           0.9492
Neg Pred Value            0.9669   